In [1]:
from config import *

from easyric.caas_lite import TiffSpliter
from easyric.objects import Pix4D
from easyric.io import shp, geotiff, plot
from easyric.calculate import geo2raw, geo2tiff

import os
import shapefile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import tifffile



In [2]:
import re
import json
from scipy.spatial import KDTree
from easyric import caas_lite
import tifffile as tf
from skimage.transform import ProjectiveTransform

In [80]:
import random
from tqdm import tqdm

In [50]:
#shp_file = r"Y:\hwang_Pro\data\2020_tanashi_broccoli\02_GIS\rotate_grids\split_grid_2.5m.shp"
shp_file = r"Y:\hwang_Pro\data\2021_tanashi_broccoli\02_GIS\split_grid.shp"

process_area = shp.read_shp2d(shp_file, name_field="id")
process_area.keys()

[io][shp][fields] Shp fields: {'id': 0, 'left': 1, 'top': 2, 'right': 3, 'bottom': 4}
[io][shp][proj] find ESRI projection file Y:\hwang_Pro\data\2021_tanashi_broccoli\02_GIS\split_grid.prj, and successfully obtain projection cartesian


dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '155', '156', '157', '158', '159', '160', '161', '162', '163', '

In [51]:
process_area["106"]

array([[ 368116.56276649, 3956027.70571439],
       [ 368119.00813549, 3956028.22549361],
       [ 368119.52791472, 3956025.78012461],
       [ 368117.08254572, 3956025.26034538],
       [ 368116.56276649, 3956027.70571439]])

In [57]:
root = f"Y:/hwang_Pro/data/2021_tanashi_broccoli/12_locate_by_yolo/sorted_id.shp"
points = shp.read_shp2d(root, name_field="name")

[io][shp][fields] Shp fields: {'name': 0}
[io][shp][proj] could not find ESRI projection file Y:/hwang_Pro/data/2021_tanashi_broccoli/12_locate_by_yolo/sorted_id.prj, could not operate auto-convention, Please convert projection system manually.


In [65]:
id_list = []
points_np = np.zeros((0,2))
for k, p in points.items():
    id_list.append(int(k))
    points_np = np.vstack([points_np, p])

In [67]:
kdtree = KDTree(points_np)

# Prepare pix4d project data

In [23]:
todo_pool = ["210512", "210514", "210515", "210519", "210520", "210526"]
date_dict = {}
for tp in todo_pool:
    p2 = Paths(tp, year=2021)
    p4d = Pix4D(project_path=p2.pix4d_project, 
                raw_img_path=p2.raw_img, 
                project_name=p2.project_name,
                param_folder=p2.pix4d_param)
    
    before_project = pd.read_csv(f"{p2.root}/13_roi_on_raw/{p2.project_name}.csv")
    
    date_dict[tp] = {"p2":p2, "p4d":p4d, "bp":before_project}

[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]
[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]
[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]
[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]
[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]
[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 54N|WGS 84|'] to geotiff coordinate tag [WGS 84 / UTM zone 54N]
[io][geotiff][GeoCorrd] Comprehense [* 34737 geo_ascii_params (30s) b'WGS 84 / UTM zone 

In [5]:
def read_json(json_path):
    container = []
    with open(json_path, "r", encoding="utf-8") as f:
        js = json.loads(f.read())
        for item in js["shapes"]:
            points = np.asarray(item["points"])
            points = np.append(points, points[0,:][None,:], axis=0)
            container.append(points)
    return container


def json_name(date, roi_id, file_name):
    return f"labeled_v0_{date}_{roi_id}_{file_name[:-4]}.json"


def mk_rm_line(bound_np, rm_bound_id):
    bound_np = bound_np[:-1, :]
    
    return shapely.geometry.LineString(bound_np[rm_bound_id-1:rm_bound_id+2,:])


def get_grid_roi_info(df_csv, id, image_name):
    filtered = df_csv[(df_csv.id == id) & (df_csv.image == image_name)]
    roi_str = filtered.roi.values[0]
    roi_str = roi_str.replace("\n ", ",")
    roi_str = re.sub("\s+", ",", roi_str)
    roi_np = np.asarray(eval(roi_str))
    off_x = filtered.offset_x.values[0]
    off_y = filtered.offset_y.values[0]
    
    return roi_np, off_x, off_y

In [74]:
def find_polygon_center(poly):
    xmin, ymin  = poly.min(axis=0)
    xmax, ymax  = poly.max(axis=0)
    
    xctr = (xmin + xmax) / 2
    yctr = (ymin + ymax) / 2
    
    return np.asarray([xctr, yctr])

In [6]:
json_folder = r"Y:\hwang_Pro\data\2021_tanashi_broccoli\13_roi_on_raw\pred"
json_files = os.listdir(json_folder)

In [12]:
json_files[0][:-5].split("_")

['pred', '20210512', '1', 'DJI', '0329']

In [90]:
total_result = {}
div_thresh = 0.20 # m

for js_file in tqdm(json_files):
    js_path = os.path.join(json_folder, js_file)
    
    _, date, plot_id, _, photo = js_file[:-5].split("_")
    date = date[2:]
    photo = f"DJI_{photo}.JPG"
    
    p2 = date_dict[date]["p2"]
    p4d = date_dict[date]["p4d"]
    before_project = date_dict[date]["bp"]
    
    if date not in total_result.keys():
        total_result[date] = {}
    
    roi_np, off_x, off_y = get_grid_roi_info(before_project, int(plot_id), photo)
    detect_poly = read_json(js_path)
    
    # correct json offsets
    detect_poly_correct = []
    for roi in detect_poly:
        # correct by distortion
        roi_off = roi + np.asarray([off_x, off_y])
        detect_poly_correct.append(roi_off)
    
    # filter in broccolis
    grid_poly = shapely.geometry.Polygon(roi_np)
    rm_line = mk_rm_line(roi_np, 2)
    
    shapely_detect_in = []
    shapely_detect_out = []
    for i, roi in enumerate(detect_poly_correct):
        roi_spy = shapely.geometry.Polygon(roi)
        if roi_spy.intersects(rm_line):
            shapely_detect_out.append(i)
        else:
            if roi_spy.intersects(grid_poly):
                shapely_detect_in.append(i)
            else:
                shapely_detect_out.append(i)
                
    # reverse to dom coords
    pt = ProjectiveTransform()
    pt.estimate(process_area[plot_id] - p4d.offset.np[0:2], roi_np)
    
    ## crop dom to small parts 
    new_coord = geotiff.geo2pixel(process_area[plot_id], geo_head=p4d.dom_header)
    cropped, np_off, offset = geotiff.crop_by_coord(p4d.dom_file, new_coord, buffer=20)
    plt.imshow(cropped)
    
    new = new_coord - offset
    plt.plot(*new.T, 'b--')
    
    save_img = False
    for idx in shapely_detect_in:
        inv_np = pt.inverse(detect_poly_correct[idx])
        
        inv_np_geo = inv_np + p4d.offset.np[0:2]
        
        
        dist, id_list_idx = kdtree.query(find_polygon_center(inv_np_geo))
        broccoli_id = id_list[id_list_idx]
        
        if dist > div_thresh:
            color = "g"
            print(f"Broccoli {broccoli_id} detect distence {dist} is over {div_thresh}")
            save_img = True
        else:
            color = "r"
            
        if broccoli_id in total_result[date].keys():
            total_result[date][broccoli_id].append(inv_np_geo)
        else:
            total_result[date][broccoli_id] = [inv_np_geo]
                
        # plot
        inv_np_dom = geotiff.geo2pixel(inv_np_geo, geo_head=p4d.dom_header) - offset
        plt.plot(*inv_np_dom.T, f"{color}-")
        plt.gcf().set_size_inches((16, 16))
        
    # decide which to save
    if random.randint(0, 10) == 5 or save_img:
        plt.savefig(f"Y:/hwang_Pro/data/2021_tanashi_broccoli/20_results/01_project_on_dom/{save_img}_{date}_{plot_id}.png")
    plt.clf()

  0%|          | 0/1008 [00:00<?, ?it/s]

Broccoli 2932 detect distence 0.3114719540487472 is over 0.2
Broccoli 1705 detect distence 0.21497188846024154 is over 0.2
Broccoli 1193 detect distence 0.23470977543744515 is over 0.2
Broccoli 941 detect distence 0.20591882560904776 is over 0.2
Broccoli 1706 detect distence 0.2233823042128155 is over 0.2
Broccoli 1493 detect distence 0.2220009607005008 is over 0.2
Broccoli 1491 detect distence 0.2316677634267242 is over 0.2
Broccoli 1637 detect distence 0.20653894474271794 is over 0.2
Broccoli 1502 detect distence 0.23028037894899245 is over 0.2
Broccoli 2146 detect distence 0.355721071505166 is over 0.2
Broccoli 2181 detect distence 0.3342462727280428 is over 0.2
Broccoli 2184 detect distence 0.30356863866252265 is over 0.2
Broccoli 2185 detect distence 0.3095344602076526 is over 0.2
Broccoli 2182 detect distence 0.3131831430884787 is over 0.2
Broccoli 2188 detect distence 0.32125658006624297 is over 0.2
Broccoli 2047 detect distence 0.21559579506141197 is over 0.2
Broccoli 2048 dete

<Figure size 1152x1152 with 0 Axes>

# region props

In [150]:
from skimage.measure import label, regionprops, regionprops_table

In [177]:
def draw_binary_image(poly_list):
    poly_merge = np.vstack(poly_list)
    
    xmax, ymax = np.max(poly_merge, axis=0)
    xmin, ymin = np.min(poly_merge, axis=0)
    xlen = xmax - xmin
    ylen = ymax - ymin

    res = 0.001 # 1mm/ pix
    
    w = xlen / res
    h = ylen / res

    im = Image.new(mode='1', size=tuple(np.ceil([w, h]).astype(int)))
    draw = ImageDraw.Draw(im)
    
    for p in poly_list:
        points = (p - np.asarray([xmin, ymin])) / res
        draw.polygon(points.reshape(len(points)*2).tolist(), fill='white', outline='white')
        
    return np.asarray(im)

def remove_outlier_poly(poly_list, broccoli_id, points):
    
    center = points[str(broccoli_id)]
    
    dist_list = []
    area_list = []
    for poly in poly_list:
        pc = find_polygon_center(poly)
        dist = calc_dist(pc, center[0, :])
        dist_list.append(dist)
        
        xmin, ymin = np.min(poly, axis=0)
        #area = shapely.geometry.Polygon(poly - np.asarray([xmin, ymin])).area ** 10000 # cm2
        #area_list.append(area)
        
    dist_np = np.asarray(dist_list)
    #area_list = np.asarray(area_list)
    
    """
    dist_np_div = dist_np - dist_np.min()
    
    selected = dist_np_div < 0.15
    
    new_list = []
    for i, b in enumerate(selected):
        if b:
            new_list.append(poly_list[i])
            
    return new_list
    """
    
    return [poly_list[dist_np.argmin()]]
    
    
def calc_dist(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

In [178]:
props_all = pd.DataFrame(columns=["date", "label", "area", "convex_area", "eccentricity", "equivalent_diameter", "major_axis_length", "minor_axis_length", "perimeter", 'circularity'])

for date, v in tqdm(total_result.items(), desc="date"):
    
    for broccoli_id, poly_list in tqdm(v.items(), desc="id"):
        
        poly_num = len(poly_list)
        
        if poly_num == 1:
            binary_img = draw_binary_image(poly_list)
        else:
            # draw plots
            fig, ax = plt.subplots(1,2, dpi=300)
            
            p4d = date_dict[date]["p4d"]
            
            new_coord = geotiff.geo2pixel(np.vstack(poly_list), geo_head=p4d.dom_header)
            cropped, np_off, offset = geotiff.crop_by_coord(p4d.dom_file, new_coord, buffer=20)
            ax[0].imshow(cropped)
            
            for p in poly_list:
                pix = geotiff.geo2pixel(p, geo_head=p4d.dom_header) - offset
                ax[0].plot(*pix.T, 'r-', alpha=0.3)
                
            center = points[str(broccoli_id)]
            center_pix = geotiff.geo2pixel(center, geo_head=p4d.dom_header) - offset
            ax[0].plot(center_pix[0, 0], center_pix[0, 1], 'ro')
            
            # judge if noise -> clean poly_list
            poly_list = remove_outlier_poly(poly_list, broccoli_id, points)
            binary_img = draw_binary_image(poly_list)

            for p in poly_list:
                pix = geotiff.geo2pixel(p, geo_head=p4d.dom_header) - offset
                ax[0].plot(*pix.T, 'r-', alpha=0.7)
                
            ax[1].imshow(binary_img, cmap="gray")
            ax[1].invert_yaxis()

            plt.savefig(f"Y:/hwang_Pro/data/2021_tanashi_broccoli/20_results/02_fix_wrong/{date}_{broccoli_id}.png")
            plt.close(fig)
            
        label_img = label(binary_img)
        props = pd.DataFrame(regionprops_table(label_img, properties=["area", "convex_area", "eccentricity", "equivalent_diameter", "major_axis_length", "minor_axis_length", "perimeter"]))
        props['circularity'] = 4 * props.area * np.pi / props.perimeter ** 2
        props['date'] = f"20{date}"
        props['label'] = broccoli_id
        props = props[["date", "label", "area", "convex_area", "eccentricity", "equivalent_diameter", "major_axis_length", "minor_axis_length", "perimeter", 'circularity']]
        
        props_all.loc[len(props_all)] = props.iloc[0]

date:   0%|          | 0/6 [00:00<?, ?it/s]

id:   0%|          | 0/3293 [00:00<?, ?it/s]

id:   0%|          | 0/3423 [00:00<?, ?it/s]

id:   0%|          | 0/3282 [00:00<?, ?it/s]

id:   0%|          | 0/3195 [00:00<?, ?it/s]

id:   0%|          | 0/3103 [00:00<?, ?it/s]

id:   0%|          | 0/2925 [00:00<?, ?it/s]

In [163]:
props.iloc[0]

date                    20210512
label                        285
area                        2371
convex_area                 2529
eccentricity            0.485232
equivalent_diameter    54.944071
major_axis_length       59.21399
minor_axis_length      51.775862
perimeter               197.4386
circularity             0.764324
Name: 0, dtype: object

In [179]:
props_all

,date,label,area,convex_area,eccentricity,equivalent_diameter,major_axis_length,minor_axis_length,perimeter,circularity
0,20210512,285,2371,2529,0.485232,54.944071,59.213990,51.775862,197.438600,0.764324
1,20210512,287,2133,2470,0.537675,52.113529,58.738901,49.525839,207.515765,0.622441
2,20210512,290,3064,3156,0.436804,62.459635,66.183537,59.535859,215.823376,0.826612
3,20210512,143,2201,2310,0.507040,52.937701,57.594992,49.642407,186.610173,0.794254
4,20210512,144,2473,2582,0.654588,56.113469,65.209004,49.297071,200.852814,0.770332
...,...,...,...,...,...,...,...,...,...,...
19216,20210526,3292,35864,37599,0.525166,213.690110,233.923807,199.069049,773.050865,0.754141
19217,20210526,3289,12296,13003,0.612709,125.122953,143.472000,113.387209,456.132034,0.742664
19218,20210526,3290,5646,14470,0.921993,84.786263,240.935196,93.291666,625.428499,0.181383
19219,20210526,3291,21634,25898,0.875148,165.967660,251.979194,121.921665,681.026479,0.586163


# link to field_measure

In [180]:
link_table = pd.read_csv("out/link_table.csv")
link_table

,field,img,pos
0,26-1,1,West
1,26-2,2,West
2,26-3,3,West
3,26-4,4,West
4,26-5,5,West
...,...,...,...
3333,1-139,3618,East
3334,1-140,3619,East
3335,1-141,3620,East
3336,1-142,3621,East


In [181]:
merged_pd = pd.merge(props_all, link_table, left_on="label", right_on="img")
merged_pd = merged_pd[["date", "field", "label", "pos",  "area", "convex_area", "eccentricity", "equivalent_diameter", "major_axis_length", "minor_axis_length", "perimeter", 'circularity']]
merged_pd = merged_pd.rename(columns={"area": "area(mm2)", 
                                      "convex_area": "convex_area(mm2)", 
                                      "equivalent_diameter": "equivalent_diameter(mm)", 
                                      "major_axis_length": "major_axis_length(mm)",
                                      "minor_axis_length": "minor_axis_length(mm)", 
                                      "perimeter": "perimeter(mm)"})
merged_pd

,date,field,label,pos,area(mm2),convex_area(mm2),eccentricity,equivalent_diameter(mm),major_axis_length(mm),minor_axis_length(mm),perimeter(mm),circularity
0,20210512,24-2,285,West,2371,2529,0.485232,54.944071,59.213990,51.775862,197.438600,0.764324
1,20210514,24-2,285,West,4424,4565,0.438728,75.052060,79.395531,71.346405,260.350288,0.820179
2,20210515,24-2,285,West,5680,5766,0.314625,85.041170,87.950560,83.484087,291.521861,0.839877
3,20210519,24-2,285,West,12854,13470,0.500925,127.930532,139.130930,120.416505,460.315801,0.762318
4,20210520,24-2,285,West,14110,14488,0.489623,134.035107,144.629077,126.107069,460.658946,0.835560
...,...,...,...,...,...,...,...,...,...,...,...,...
17697,20210526,15-125,1660,West,4380,4698,0.611704,74.677903,84.545288,66.882632,284.971609,0.677768
17698,20210526,12-66,2015,East,4128,4778,0.895483,72.497813,117.143445,52.140058,314.699531,0.523790
17699,20210526,4-21,3082,East,9414,10683,0.893977,109.481857,170.052081,76.202503,446.931024,0.592247
17700,20210526,1-33,3513,East,4057,5397,0.942323,71.871641,137.165555,45.910162,340.693434,0.439225


In [182]:
merged_pd.to_excel("out/2021_measured.xlsx", index=False)